In [70]:
###############################################################################
# Intense QC for DAFNI workflow
# Amy Green, Robin Wardle
# July 2023
###############################################################################

###############################################################################
# Python libraries
###############################################################################
from os.path import join, exists
import os, sys
import numpy as np
import pandas as pd
import logging
import pathlib

In [71]:
###############################################################################
# Constants
###############################################################################
QC_SUCCESS_FILENAME = "success"
QC_LOG_FILENAME = "intense-qc.log"

In [72]:
###############################################################################
# Paths
###############################################################################

# Output paths to save files
platform = os.getenv("INTENSE_QC_ENV")
if platform=="docker":
    data_path = os.getenv("DATA_PATH", "/data")
else:
    data_path = os.getenv("DATA_PATH", "./data")
input_path = os.path.join(data_path, "inputs")
output_path = os.path.join(data_path, "outputs")
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

# Reset the success and log files
if os.path.isfile(os.path.join(output_path, QC_SUCCESS_FILENAME)):
    os.remove(os.path.join(output_path, QC_SUCCESS_FILENAME))
if os.path.isfile(os.path.join(output_path, QC_LOG_FILENAME)):
    os.remove(os.path.join(output_path, QC_LOG_FILENAME))


In [73]:
###############################################################################
# Logging
###############################################################################
# Configure logging
logging.basicConfig()
logging.root.setLevel(logging.INFO)

# Logging instance
#logger = logging.getLogger(pathlib.PurePath(__file__).name)
logger = logging.getLogger("intense-qc")
logger.propagate = False

# Console messaging
console_formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')
console_handler = logging.StreamHandler(stream=sys.stdout)
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# File logging
file_formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s')
file_handler = logging.FileHandler(output_path / pathlib.Path(QC_LOG_FILENAME))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

logger.info("Logger initialised")

# Some additional logging info
logger.info("data_path = {}".format(data_path))
logger.info("output_path = {}".format(output_path))

INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:Logger initialised
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:data_path = ./data
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs
INFO:intense-qc:output_path = ./data/outputs


In [74]:
###############################################################################
# Other Parameters
#   Remember that parameters are passed as environment variables,
#   i.e. they are strings and will need to be converted
###############################################################################

# Dates for files
start_date = os.getenv("RUN_START_DATE", "2023-06-20")
end_date = os.getenv("RUN_END_DATE", "2023-06-30")

# Bounding box for data
# e_l, n_l, e_u, n_u = [355000, 534000, 440000, 609000]
try:
    e_l = int(os.getenv("BB_E_L", "355000"))
    n_l = int(os.getenv("BB_N_L", "534000"))
    e_u = int(os.getenv("BB_E_U", "440000"))
    n_u = int(os.getenv("BB_N_U", "609000"))
    bbox = [e_l, e_u, n_l, n_u]
except (TypeError, ValueError, Exception) as e:
    logger.error("Error converting environmental parameters: {}".format(e))
    raise


In [75]:
###############################################################################
# Quality control test data library path
# TODO: This lib needs to be incorporated as a sub-repo, not as copied files
###############################################################################
static_data_path = join(r"./", "static")
intense_path = join(static_data_path, "intense-qc")
etccdi_data_path = join(join(intense_path, "tests"), "etccdi_data")

# Adds intense folder location to path so can run correct version
os.sys.path.append(intense_path)
from intense import gauge, qc, utils
from pyproj import Transformer

In [76]:
def get_gauge_flags(station_id, data, loc):
    '''
    Function to apply wokring Intense QC hourly checks on gauge data. 
    Args: data is the gauge data (pd.Series), loc is the gauge location (eastings, northings)
    Returns: flags as a dictionary {gauge flagged, years flagged, obs flagged}
    '''
    eastings, northings = loc
    # convert coordinates to lat/lon
    transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
    latitude, longitude = transformer.transform(eastings, northings)
    
    # create gauge object
    rain_gauge = gauge.Gauge(
        station_id,
        path_to_original_data="",
        latitude=latitude,
        longitude=longitude,
        original_timestep="15min",
        original_units="mm/h",
        new_units="mm/h",
        new_timestep="1h",
        data=data
    )
    rain_gauge.get_info()

    # create qc object 
    test = qc.Qc(
        gauge=rain_gauge,
        etccdi_data_folder=etccdi_data_path
    )

    # checks that don't work on existing data
    """
    test.check_percentiles()
    test.check_k_largest()
    test.check_intermittency()
    test.change_in_min_val_check() 
    test.cwd_check() # missing function in utils file???
    test.change_in_min_val_check() # Change in minimum value check, homogeneity check to see if the resolution of the data has changed. Change flag, flag years
    test.find_neighbours("hourly") # frequency: must be either hourly, daily or monthly, Names or names and paths of neighbouring stations
    # conditions are: must be within 50km, at least 3 years overlap, select the closest 10, don't have three years of data
    # check_hourly_neighbours(), check_daily_neighbours(), check_monthly_neighbours()
    test.get_flags() # runs all checks, fails at find_neighbours()
    """

    ### run checks ###

    flagged_sdii = 0
    sdii_thresh = 100 # just arbitrary atm
    if any(np.array(test.get_sdii()) > sdii_thresh): # Simple precipitation intensity index, SDII from ETCCDI and from gauge values (sdii_gridded, sdii_gauge), not sure how to use this
        flagged_sdii = 1
    
    # Flag data if any of these don't return 0:
    gauge_checks = [
        test.check_days_of_week(), # Checks if proportions of rainfall in each day is significantly different
        test.check_break_point(), # Pettitt breakpoint check
        flagged_sdii
    ]
    
    flagged_gauge = sum(gauge_checks) != 0
    #if flagged_gauge:
    #    print("Gauge", np.array(["days of week", "break point", "sdii"])[np.array(gauge_checks) == 1])
    #    print(test.check_break_point())

    # Flag individual data observations if don't return 0:
    obs_checks = pd.DataFrame(index=rain_gauge.data.index)
    obs_checks["world_record"] = test.world_record_check_ts() # Checks if and to what degree the world record has been exceeded by each rainfall value, 4, 3, 2 or 1 if exceeded by > 1.5x, 1.33x, 1.22x or 0x respectively and 0 if not exceeded for each value
    obs_checks["rx1day"] = test.rx1day_check_ts() # Checks hourly values against maximum 1-day precipitation, Magnitudes of exceedance for each day
    obs_checks["cdd"] = test.cdd_check() # ETCCDI provide an index for maximum length of dry spell. Look for suspicious number of consecutive dry hours recorded. Consecutive Dry Days: Maximum length of dry spell, maximum number of consecutive days with RR < 1mm. Magnitudes of exceedence of the length of longest dry period
    obs_checks["daily_accums"] = test.daily_accums_check() # Check daily accumulations. Suspect daily accumulations flagged where a recorded rainfall amount at these times is preceded by 23 hours with no rain. A threshold of 2x the mean wet day amount for the corresponding month is applied to increase thechance of identifying accumulated values at the expense of genuine, moderate events
    obs_checks["monthly_accums"] = test.monthly_accums_check() # Check monthly accumulations. Flags month prior to high value
    obs_checks["streaks"] = test.streaks_check() # Streaks: This is where you see the same value repeated in a run. Currently this records streaks of 2hrs in a row or more over 2 x Monthly mean rainfall. It is considered to be unlikely that you would see even 2 consecutive large rainfall amounts. For this code I have substituted the monthly mean rainfall for SDII as I want the thresholds to be independent of the rainfall time series as the global dataset is of highly variable quality.
    flagged_obs = obs_checks.index[obs_checks.sum(1) > 0]

    # Flags each individual year:
    year_flags = np.array([
        test.r99ptot_check_annual(), # Check against R99pTOT: R99pTOT. Annual total PRCP when RR > 99p. Magnitudes of exceedance for yearly 99th percentiles
        test.prcptot_check_annual() # check against annual total: PRCPTOT. Annual total precipitation in wet days. Magnitudes of exceedance for yearly totals
    ])

    flagged_years = np.arange(rain_gauge.data.index.min().year, rain_gauge.data.index.max().year + 1)[year_flags.sum(0) < 0]

    return({"gauge" : flagged_gauge, "years" : flagged_years, "obs" : flagged_obs})

In [77]:
gauge_sources = ["EA", "UO", "CS", "NGIF"]

for source in gauge_sources:

    source_input_filepath = join(input_path, source)
    if not os.path.isdir(source_input_filepath):
        continue

    source_output_filepath = join(output_path, source)
    os.makedirs(source_output_filepath, exist_ok=True)

    for f in os.listdir(source_input_filepath):

        if f.endswith(".csv"):
            gauge_15min = pd.read_csv(join(source_input_filepath, f), index_col=0)
            gauge_15min.index = pd.to_datetime(gauge_15min.index, utc=True)

            gauge_1hr = gauge_15min.resample("1h").mean()
            station_id, eastings, northings = f.split(".")[0].split("_")
    
            loc = (eastings, northings)
            flags = get_gauge_flags(station_id, gauge_1hr.dropna().iloc[:, 0], loc)
    
            # Only include gauge data that is not flagged
            if not flags["gauge"]:
                # currenly ignoring year flag as not enough data

                # remove flagged observationns
                if len(flags["obs"]) > 0:
                    cond = [idx in flags["obs"] for idx in gauge_15min.index.round("1h")]
                    gauge_15min.loc[cond] = np.nan

                gauge_15min.to_csv(join(source_output_filepath, f))


TypeError: __init__() missing 1 required positional argument: 'station_id'

In [ ]:
###############################################################################
# Conclusion
###############################################################################
logger.info("Completed successfully")
os.system("cd " + output_path + "; touch success" )